In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

from FRLM import network

from matplotlib import pyplot as plt
import matplotlib as mpl
from network2tikz import plot
import pandas as pd
import numpy as np
import json
import compress_json

network_name = '25NODE'
solution_fn = 'results/test/25NODE_10_15_25.json.gz'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
json_data = compress_json.load(solution_fn)

params = json_data['params']
solution = json_data['solution']

net = network.Net(dataDir=f"instance/{params['network_name']}", dist_multiplier=params['dist_multiplier'])

node_weight_l = [[i,net.G.nodes[i]['weight']] for i in net.G.nodes if net.G.nodes[i]['weight'] > 0]
weighted_nodes = [i for i,w in sorted(node_weight_l,key=lambda l:l[1], reverse=True)][:params['num_node']]

arcs = {}
for k,(p,l,t,f) in json_data['covered_full_paths_k'].items():
    prev_i = p[0]
    for i in p[1:]:
        arc = tuple(sorted((prev_i, i)))
        if arc not in arcs:
            arcs[arc] = f
        else:
            arcs[arc] += f
        prev_i = i

layout = {i:net.G.nodes[i]['pos'] for i in net.G.nodes}

visual_style = {}

visual_style['node_label_off'] = False
visual_style['node_opacity'] = [0.5 for i in net.G.nodes]
visual_style['node_label'] = [net.G.nodes[i]['weight'] if i in weighted_nodes else '' for i in net.G.nodes]
# visual_style['node_label_color'] = ['white' for i in net.G.nodes]
visual_style['node_size'] = [0.4 if i in weighted_nodes else 0.2 for i in net.G.nodes]
visual_style['node_color'] = []
visual_style['node_shape'] = []
for i in net.G.nodes:
    if i in solution[0]:
        if solution[1][solution[0].index(i)] == 1:
            shape = 'diamond'
            visual_style['node_size'][i] = 0.5
        else:
            shape = 'circle'
        color = 'red'
    else:
        color = 'white!50!gray'
        shape = 'circle'
    visual_style['node_color'].append(color)
    visual_style['node_shape'].append(shape)

visual_style['edge_width'] = []
visual_style['edge_color'] = []
for (i1,i2) in net.G.edges:
    if i1 > i2:
        continue
    if (i1,i2) in arcs:
        visual_style['edge_width'].append(max(arcs[(i1,i2)]/250, 1))
        visual_style['edge_color'].append('red')
    else:
        visual_style['edge_width'].append(1)
        visual_style['edge_color'].append('black')

visual_style['edge_opacity'] = 0.8
visual_style['edge_label'] = [net.G.edges[i]['weight'] for i in net.G.edges]
visual_style['edge_label_color'] = ['black' for i in net.G.edges]
visual_style['edge_label_position'] = ['below' for i in net.G.edges]

visual_style['canvas'] = (18,18)
visual_style['margin'] = 1

visual_style['layout'] = layout

plot(net.G, 'solution.pdf', **visual_style)
# plot(net.G, 'figure/TEXAS_solution.csv', **visual_style)